In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18
import numpy as np
import random

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# Loading CIFAR-10 dataset
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
target_tensor = torch.tensor(full_train_dataset.targets)

class_indices = [torch.where(target_tensor == i)[0][:25] for i in range(2)]
class_indices = torch.cat(class_indices)

# Train dataset and loader
train_dataset = Subset(full_train_dataset, class_indices)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

# # Test dataset and loader
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Modify ResNet to fit the small data scenario
def create_small_resnet(num_classes=2, dropout_rate=0.5):
    model = resnet18(pretrained=False)
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(num_features, num_classes)
    )
    return model

# Initialize the model
model = create_small_resnet().to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)

            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')
        scheduler.step()

# Train the model
train_model(model, train_loader, criterion, optimizer)


Epoch [1/50], Loss: 1.0799
Epoch [2/50], Loss: 0.7490
Epoch [3/50], Loss: 0.8518
Epoch [4/50], Loss: 0.6642
Epoch [5/50], Loss: 0.5490
Epoch [6/50], Loss: 0.1877
Epoch [7/50], Loss: 0.3505
Epoch [8/50], Loss: 0.2533
Epoch [9/50], Loss: 0.1633
Epoch [10/50], Loss: 0.5683
Epoch [11/50], Loss: 0.1519
Epoch [12/50], Loss: 0.0322
Epoch [13/50], Loss: 0.1378
Epoch [14/50], Loss: 0.4867
Epoch [15/50], Loss: 0.0818
Epoch [16/50], Loss: 0.2484
Epoch [17/50], Loss: 0.2487
Epoch [18/50], Loss: 0.1785
Epoch [19/50], Loss: 0.1007
Epoch [20/50], Loss: 0.0436
Epoch [21/50], Loss: 0.1245
Epoch [22/50], Loss: 0.0227
Epoch [23/50], Loss: 0.0296
Epoch [24/50], Loss: 0.0352
Epoch [25/50], Loss: 0.0088
Epoch [26/50], Loss: 0.0177
Epoch [27/50], Loss: 0.0172
Epoch [28/50], Loss: 0.1984
Epoch [29/50], Loss: 0.0099
Epoch [30/50], Loss: 0.0414
Epoch [31/50], Loss: 0.0577
Epoch [32/50], Loss: 0.0494
Epoch [33/50], Loss: 0.0202
Epoch [34/50], Loss: 0.0126
Epoch [35/50], Loss: 0.0113
Epoch [36/50], Loss: 0.0193
E

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Inference mode, gradient is not needed
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model
accuracy = evaluate_model(model, test_loader)
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')


Accuracy of the model on the test images: 15.06%
